In [3]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd

In [6]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[260:] 

In [7]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoEditer(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    '''
    xMax = max((data['x']).astype(int))
    xMin = min((data['x']).astype(int))
    yMax = max((data['y']).astype(int))
    yMin = min((data['y']).astype(int))
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
#     if yMax - yMin < xMax - xMin:
#         length = yMax - yMin
#     else:
#         length = xMax - xMin
    
    
    return xMin, yMin, lengthX, lengthY



In [ ]:
videoEditer('1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4", 10,10,300)

In [ ]:
x, y, lengthX, lengthY = getCoord(move_data)

In [ ]:
sorted(move_data['y'], reverse = True)

In [ ]:
move_data

In [ ]:
x, y, lengthX, lengthY

In [ ]:
videoEditer('1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4", x, y, lengthX, lengthY)

In [5]:
p = cv2.imread("test2.jpg")
p[:,:,:1]


NameError: name 'cv2' is not defined

In [8]:
p = cv2.imread("test2.jpg")
p = p[:,:,::-1]
# p = cv2.cvtColor(p[:,:,::-1], cv2.COLOR_BGR2GRAY)
# p = cv2.cvtColor(p, cv2.COLOR_GRAY2BGR)
cv2.imwrite("test3.jpg",p[0:200,0:150])

True

In [23]:
from moviepy.video.fx import crop
from moviepy.editor import VideoFileClip

video = VideoFileClip('1034 SI_B, Aug 15, 13 7 49.mp4')
clip = crop.crop(video, x1=0, y1=0, x2=200, y2=275)
clip.write_videofile("test9.mp4")


                                                                  
                                                               

t:   0%|          | 1/3481 [08:35<498:00:15, 515.18s/it, now=None]

                                                                  
                                                               

t:   0%|          | 1/3481 [08:35<498:01:47, 515.20s/it, now=None]

t:   0%|          | 1/3481 [07:46<451:23:50, 466.96s/it, now=None]


t:   0%|          | 0/3481 [00:00<?, ?it/s, now=None]


t:   1%|          | 29/3481 [00:00<00:12, 287.27it/s, now=None]

Moviepy - Building video test9.mp4.
Moviepy - Writing video test9.mp4






t:   2%|▏         | 54/3481 [00:00<00:12, 273.07it/s, now=None]


t:   2%|▏         | 68/3481 [00:00<00:16, 211.92it/s, now=None]


t:   3%|▎         | 93/3481 [00:00<00:15, 221.59it/s, now=None]


t:   3%|▎         | 119/3481 [00:00<00:14, 229.21it/s, now=None]


t:   4%|▍         | 147/3481 [00:00<00:13, 241.78it/s, now=None]


t:   5%|▌         | 176/3481 [00:00<00:13, 254.18it/s, now=None]


t:   6%|▌         | 202/3481 [00:00<00:12, 255.45it/s, now=None]


t:   7%|▋         | 234/3481 [00:00<00:11, 270.66it/s, now=None]


t:   8%|▊         | 270/3481 [00:01<00:11, 291.19it/s, now=None]


t:   9%|▊         | 301/3481 [00:01<00:10, 294.10it/s, now=None]


t:  10%|▉         | 332/3481 [00:01<00:10, 297.37it/s, now=None]


t:  10%|█         | 364/3481 [00:01<00:10, 299.05it/s, now=None]


t:  11%|█▏        | 394/3481 [00:01<00:10, 296.08it/s, now=None]


t:  12%|█▏        | 426/3481 [00:01<00:10, 299.62it/s, now=None]


t:  13%|█▎        | 458/3481 [00:01<00:09, 304.38it/s, now=Non

t:  99%|█████████▉| 3454/3481 [00:11<00:00, 296.20it/s, now=None]


                                                                 

                                                                  
                                                               

t:   0%|          | 1/3481 [08:47<509:40:58, 527.26s/it, now=None]

                                                                  
                                                               

t:   0%|          | 1/3481 [08:47<509:41:40, 527.27s/it, now=None]

Moviepy - Done !
Moviepy - video ready test9.mp4


In [ ]:
import imutils
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread("test2.jpg")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
 
 # show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [ ]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 1) * ratio)
 
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
 
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

In [ ]:
video = videoWriter("crop", '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4")
cap = video[0]
out = video[1]
